In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import sys 
import os
sys.path.append(os.path.abspath("/media/sf_Project2/Code"))
from IO import Input
from IO import Output
import pandas as pd

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [2]:
with cd("/media/sf_Project2/Code"): 
    train_dataset = np.array(Input.load_trainset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    train_labels = np.array(Input.load_trainset_labels()).astype('float32')
    valid_dataset = np.array(Input.load_validationset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    valid_labels = np.array(Input.load_validationset_labels()).astype('float32')

In [3]:
num_labels=10
train_labels = np.squeeze((np.arange(num_labels) == train_labels[:,None]).astype(np.float32))
valid_labels = np.squeeze((np.arange(num_labels) == valid_labels[:,None]).astype(np.float32))
train_labels0 = train_labels[:,1]
train_labels0 = train_labels0.reshape((train_labels.shape[0],1))

print(train_labels0)
#print(train_dataset)
#train_labels = train_labels.reshape((train_labels.shape[0],1))
#valid_labels = valid_labels.reshape((valid_labels.shape[0],1))
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
feature_size = train_dataset.shape[1]
print(train_labels)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Training set (14363, 3983) (14363, 10)
Validation set (8061, 3983) (8061, 10)
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1))
          / predictions.shape[0])

# Network:

In [1]:
batch_size = 64
hlSize0 = 1024
beta = 0.004
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 200
decay_rate = 0.98
learningStart=0.00007

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,num_labels], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    return layer2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights)
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

NameError: name 'tf' is not defined

In [2]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  valid_prediction_val = valid_prediction.eval()

NameError: name 'tf' is not defined

In [12]:
validData = pd.DataFrame(valid_prediction_val)
Output.to_outputfile(validData,1,'NNSTRUCTURE2valid')

In [ ]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000),'RF',100)).astype('float32')

In [ ]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(firstHalfTest)

In [ ]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000,60000),'RF',100)).astype('float32')

In [ ]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

In [ ]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(60000,80000),'RF',100)).astype('float32')

In [ ]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

In [ ]:
testClass = np.concatenate([firstHalfTest,secondHalfTest],0)
testClass = pd.DataFrame(testClass)
Output.to_outputfile(testClass,1,"NNSTRUCTURE1testset",validation=False)